⚠️ The original notebook is publicly available on Databricks : [view notebook](
https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/8772860121263641/3904133659262707/2895445563804111/latest.html)

# Importing dataset

In [ ]:
filepath = 's3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json'

from pyspark.sql import functions as F

data_json = spark.read.json(filepath)
data_json.printSchema()
data_json.count()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

# Exploding ALL except TAGS & CATEGORIES

In [ ]:
flat_data_no_tags = data_json.withColumn('appid', F.col("data.appid")) \
        .withColumn('categories', F.col("data.categories")) \
        .withColumn('ccu', F.col("data.ccu")) \
        .withColumn('developer', F.col("data.developer")) \
        .withColumn('discount', F.col("data.discount")) \
        .withColumn('genre', F.col("data.genre")) \
        .withColumn('header_image', F.col("data.header_image")) \
        .withColumn('initialprice', F.col("data.initialprice")) \
        .withColumn('languages', F.col("data.languages")) \
        .withColumn('name', F.col("data.name")) \
        .withColumn('negative', F.col("data.negative")) \
        .withColumn('owners', F.col("data.owners")) \
        .withColumn('platforms_linux', F.col("data.platforms.linux")) \
        .withColumn('platforms_mac', F.col("data.platforms.mac")) \
        .withColumn('platforms_windows', F.col("data.platforms.windows")) \
        .withColumn('positive', F.col("data.positive")) \
        .withColumn('price', F.col("data.price")) \
        .withColumn('publisher', F.col("data.publisher")) \
        .withColumn('release_date', F.col("data.release_date")) \
        .withColumn('required_age', F.col("data.required_age")) \
        .withColumn('short_description', F.col("data.short_description")) \
        .withColumn('tags', F.col("data.tags")) \
        .withColumn('type', F.col("data.type")) \
        .withColumn('website', F.col("data.website")) \
        .drop('data') \
        .drop('tags')
flat_data_no_tags.printSchema()

root
 |-- id: string (nullable = true)
 |-- appid: long (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ccu: long (nullable = true)
 |-- developer: string (nullable = true)
 |-- discount: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- header_image: string (nullable = true)
 |-- initialprice: string (nullable = true)
 |-- languages: string (nullable = true)
 |-- name: string (nullable = true)
 |-- negative: long (nullable = true)
 |-- owners: string (nullable = true)
 |-- platforms_linux: boolean (nullable = true)
 |-- platforms_mac: boolean (nullable = true)
 |-- platforms_windows: boolean (nullable = true)
 |-- positive: long (nullable = true)
 |-- price: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- required_age: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- type: string (nullable = true)
 |-- website

# Which publisher has released the most games on Steam ?

In [ ]:
flat_data_no_tags.groupBy('publisher').count().orderBy(F.desc('count')).limit(10).toPandas()

,publisher,count
0,Big Fish Games,422
1,8floor,202
2,SEGA,165
3,Strategy First,151
4,Square Enix,141
5,Choice of Games,140
6,Sekai Project,132
7,HH-Games,132
8,,132
9,Ubisoft,127


# What are the best rated games?

In [ ]:
flat_data_no_tags.select('name', ((F.col("positive")-F.col("negative")).alias("total_score"))).orderBy(F.col("total_score").desc()).limit(10).toPandas()

,name,total_score
0,Counter-Strike: Global Offensive,5156252
1,Dota 2,1216979
2,Grand Theft Auto V,1015886
3,Terraria,992331
4,Garry's Mod,831242
5,Tom Clancy's Rainbow Six Siege,799663
6,Team Fortress 2,788984
7,Left 4 Dead 2,627008
8,Rust,620359
9,The Witcher 3: Wild Hunt,607382


# Are there years with more releases? Were there more or fewer game releases during the Covid, for example?

In [ ]:
# Let's check the date format to convert it to timestamp
flat_data_no_tags.groupBy('release_date').count().show()

+------------+-----+
|release_date|count|
+------------+-----+
|  2012/12/11|    2|
|  2019/07/22|   13|
|  2020/10/22|   52|
|  2019/07/20|    4|
|  2011/10/18|    3|
|  2012/08/13|    1|
|   2021/02/9|   11|
|   2022/11/5|    6|
|  2016/01/22|   13|
|  2013/02/21|    1|
|  2012/01/27|    1|
|   2015/04/9|   10|
|  2014/08/27|    6|
|  2015/06/22|    7|
|  2016/11/25|   14|
|  2017/10/20|   36|
|  2017/01/29|    5|
|     2017/01|    2|
|  2017/02/11|    2|
|  2007/06/12|    1|
+------------+-----+
only showing top 20 rows



In [ ]:
# Let's check all values seem well formatted
flat_data_no_tags.groupBy(F.length('release_date')).count().show()

+--------------------+-----+
|length(release_date)|count|
+--------------------+-----+
|                   9|15732|
|                   7|  123|
|                  10|39737|
|                   0|   99|
+--------------------+-----+



In [ ]:
# Let's only keep date with length = 9 & 10
flat_data_no_tags_with_date = flat_data_no_tags.filter( (F.length('release_date') == 10) | (F.length('release_date') == 9) )
flat_data_no_tags_with_date.groupBy(F.length('release_date')).count().show()

+--------------------+-----+
|length(release_date)|count|
+--------------------+-----+
|                   9|15732|
|                  10|39737|
+--------------------+-----+



In [ ]:
# Let's convert to_date() and get our stats
flat_data_no_tags_with_date = flat_data_no_tags.withColumn("release_date", F.to_date("release_date", 'yyyy/MM/d' )).withColumn('release_year', F.year(F.col("release_date")))
display(flat_data_no_tags_with_date.groupBy('release_year').count().orderBy(F.col('count').desc()).limit(10))

# The graph shows 2019 was a bad year in terms of number of games released.
# The COVID-19 outbreak started December 2019. The graph shows a regain of growth during the COVID-19 period.

release_year,count
2021,8805
2020,8287
2018,7663
2022,7451
2019,6949
2017,6006
2016,4176
2015,2566
2014,1550
2013,469


Databricks visualization. Run in Databricks to view.

# How are the prices distributed? Are there many games with a discount?

In [ ]:
display(flat_data_no_tags.select(F.col('price').cast('int')).filter(F.col('price') < 10000).collect())

price
999
999
599
1999
199
799
1299
0
299
1399


Databricks visualization. Run in Databricks to view.

In [ ]:
display(flat_data_no_tags.select(F.col('price').cast('int')).filter(F.col('price') < 1000).collect())

price
999
999
599
199
799
0
299
99
0
0


Databricks visualization. Run in Databricks to view.

### the column price might not be the games retail price given the values. Let's check initialprice

In [ ]:
display(flat_data_no_tags.select(F.col('initialprice').cast('int')).filter(F.col('initialprice') < 10000).collect())

initialprice
999
999
1999
1999
199
1999
1299
0
299
1399


Databricks visualization. Run in Databricks to view.

Initial price is in the same range of values. Let's assume it's because we don't know the currency.

### Let's check the repartition of discounts. Most games (95.5%) have no discounts.

In [ ]:
display(flat_data_no_tags.groupBy(F.col('discount')).count().orderBy(F.col("count").cast('int').desc()).collect())

discount,count
0,53173
50,350
90,239
80,228
75,223
40,164
51,146
70,137
60,137
30,131


Databricks visualization. Run in Databricks to view.

### Let's remove 0% discounts to check the repartition of discounts among discounted games.

In [ ]:
display(flat_data_no_tags.select(F.col('discount').cast('int')).filter(F.col('discount') > 0).collect())

discount
70
60
30
70
80
70
69
45
25
30


Databricks visualization. Run in Databricks to view.

# What are the most represented languages?
Languages are concatenated into a single string. Let's split them for analysis.

In [ ]:
display(flat_data_no_tags.withColumn('languages', F.explode(F.split(F.col('languages'),', '))).groupBy('languages').count().orderBy(F.col("count").desc()).limit(10))

languages,count
English,55116
German,14019
French,13426
Russian,12922
Simplified Chinese,12782
Spanish - Spain,12233
Japanese,10368
Italian,9304
Portuguese - Brazil,6750
Korean,6599


In [ ]:
display(flat_data_no_tags.withColumn('languages', F.explode(F.split(F.col('languages'),', '))).groupBy('languages').count().orderBy(F.col("count").desc()).collect())

languages,count
English,55116
German,14019
French,13426
Russian,12922
Simplified Chinese,12782
Spanish - Spain,12233
Japanese,10368
Italian,9304
Portuguese - Brazil,6750
Korean,6599


Databricks visualization. Run in Databricks to view.

# Are there many games prohibited for children under 16/18?
16, 17, 18, and 21+ games represent a very small percentage of all the games available.

In [ ]:
flat_data_no_tags.groupBy(F.col("required_age")).count().orderBy(F.col("required_age").desc()).show(21)

+------------+-----+
|required_age|count|
+------------+-----+
|      MA 15+|    1|
|           9|    1|
|           8|    3|
|          7+|    1|
|           7|    2|
|           6|    4|
|           5|    1|
|          35|    1|
|           3|    3|
|         21+|    1|
|          20|    1|
|         180|    4|
|          18|  223|
|          17|   38|
|          16|   38|
|          15|  264|
|          14|   10|
|          13|   26|
|          12|   32|
|          10|    7|
|           0|55030|
+------------+-----+



# What are the most represented genres?
Genres are concatenated into a single string. Let's split them for analysis.

In [ ]:
flat_data_no_tags.withColumn('genre', F.explode(F.split(F.col('genre'),', '))).groupBy('genre').count().orderBy(F.col("count").desc()).show()

+--------------------+-----+
|               genre|count|
+--------------------+-----+
|               Indie|39681|
|              Action|23759|
|              Casual|22086|
|           Adventure|21431|
|            Strategy|10895|
|          Simulation|10836|
|                 RPG| 9534|
|        Early Access| 6145|
|        Free to Play| 3393|
|              Sports| 2666|
|              Racing| 2155|
|Massively Multipl...| 1460|
|           Utilities|  682|
|Design & Illustra...|  406|
|Animation & Modeling|  322|
|           Education|  317|
|    Video Production|  247|
|    Audio Production|  195|
|             Violent|  168|
|   Software Training|  164|
+--------------------+-----+
only showing top 20 rows



In [ ]:
display(flat_data_no_tags.withColumn('genre', F.explode(F.split(F.col('genre'),', '))).groupBy('genre').count().orderBy(F.col("count").desc()).collect())

genre,count
Indie,39681
Action,23759
Casual,22086
Adventure,21431
Strategy,10895
Simulation,10836
RPG,9534
Early Access,6145
Free to Play,3393
Sports,2666


Databricks visualization. Run in Databricks to view.

# Are there any genres that have a better positive/negative review ratio?

In [ ]:
display(flat_data_no_tags.withColumn("score_ratio", (F.col("positive")/F.col("negative"))) \
    .withColumn('genre', F.explode(F.split(F.col('genre'),', '))) \
    .groupBy('genre').mean('score_ratio') \
    .orderBy(F.col("avg(score_ratio)").desc()) \
    .collect() \
    )

genre,avg(score_ratio)
Game Development,6.574592665282239
Adventure,6.240398840953547
Casual,6.052168411554156
Indie,5.955536577526524
RPG,5.639226703208549
Animation & Modeling,5.512636080555294
Design & Illustration,5.393380556051247
Action,5.377602748081027
Free to Play,5.291039401234781
Simulation,4.855032234213458


Databricks visualization. Run in Databricks to view.

# Do some publishers have favorite genres?
Only 4 of the big publishers (over 100 games) focus more than 50% of their games on a single genre

In [ ]:
display(flat_data_no_tags.withColumn('genre', F.explode(F.split(F.col('genre'),', '))) \
    .groupBy('publisher','genre').count() \
    .groupBy('publisher').agg( (F.max('count') / F.sum('count')).alias('ratio'), F.sum('count').alias('count') ) \
    .filter(F.col('count') > 100) \
    .orderBy(F.col("ratio").desc()) \
    .collect() \
    )

publisher,ratio,count
8floor,0.831275720164609,243
Reforged Group,0.6518518518518519,135
Slitherine Ltd.,0.6075949367088608,158
Big Fish Games,0.5011990407673861,834
Artifex Mundi,0.4866666666666667,150
Blender Games,0.4827586206896552,145
BANDAI NAMCO Entertainment,0.46846846846846846,111
Laush Studio,0.4492753623188406,276
Paradox Interactive,0.4307692307692308,130
Sokpop Collective,0.3911290322580645,248


In [ ]:
# 89 of the 468 publishers with over 10 games focus more than 50% of their games on a single genre

In [ ]:
# There are 468 publishers with more than 10 games
flat_data_no_tags.groupBy('publisher').count().filter((F.col('count') > 10)).count()

Out[208]: 468

In [ ]:
flat_data_no_tags.withColumn('genre', F.explode(F.split(F.col('genre'),', '))) \
    .groupBy('publisher','genre').count() \
    .groupBy('publisher').agg( (F.max('count') / F.sum('count')).alias('ratio'), F.sum('count').alias('count') ) \
    .filter((F.col('count') > 10) & (F.col('ratio')>0.5)) \
    .orderBy(F.col("ratio").desc()) \
    .show(90)

+--------------------+------------------+-----+
|           publisher|             ratio|count|
+--------------------+------------------+-----+
|         id Software|               1.0|   16|
|  PopCap Games, Inc.|               1.0|   19|
|        HexWar Games|0.9767441860465116|   43|
|     CITY CONNECTION|0.9473684210526315|   19|
|          grin robot|0.9090909090909091|   11|
|     HeR Interactive|0.8529411764705882|   34|
|     Notus Games Ltd|0.8461538461538461|   13|
|              8floor| 0.831275720164609|  243|
|     SNK CORPORATION|               0.8|   30|
| Ziggurat, 612 Games|0.7857142857142857|   14|
|    Arc System Works|              0.75|   36|
|       Dogenzaka Lab|              0.75|   20|
|        Sigyaad Team|              0.75|   12|
|       Sandlot Games|              0.75|   12|
|         Remix Games|              0.75|   12|
|Mens Sana Interac...|0.7407407407407407|   27|
|WASABI entertainment|0.7391304347826086|   23|
|PopCap Games, Inc...|0.7333333333333333

# What are the most lucrative genres?
Let's find the number of purchase per game. Looking at the available columns, 'owners' looks like a range of people who purchased the game.

In [ ]:
display(flat_data_no_tags.orderBy(F.col("ccu").desc()).select('name','ccu','owners').limit(20))

name,ccu,owners
Counter-Strike: Global Offensive,874053,"50,000,000 .. 100,000,000"
Dota 2,852995,"200,000,000 .. 500,000,000"
PUBG: BATTLEGROUNDS,339287,"50,000,000 .. 100,000,000"
Apex Legends,314468,"20,000,000 .. 50,000,000"
Lost Ark,273088,"20,000,000 .. 50,000,000"
Call of Duty: Modern Warfare II,206441,"2,000,000 .. 5,000,000"
Grand Theft Auto V,140671,"20,000,000 .. 50,000,000"
New World,127379,"50,000,000 .. 100,000,000"
Rust,121146,"20,000,000 .. 50,000,000"
NARAKA: BLADEPOINT,116729,"5,000,000 .. 10,000,000"


Let's get a integer from the owners column to compute the games revenu.

In [ ]:
display(flat_data_no_tags.withColumn('purchase', F.explode(F.split(F.col('owners'),' .. '))).groupBy('purchase').count().collect())

purchase,count
"50,000,000",25
"50,000",10980
0,38072
"5,000,000",432
"20,000",45357
"500,000",3095
"20,000,000",62
"1,000,000",1459
"100,000",6214
"2,000,000",861


I've interpreted the high values for prices as being the prices in cents rather than $ or €.

In [ ]:
display(flat_data_no_tags.withColumn('purchases', (F.split(F.col('owners'),' .. ')).getItem(1)) \
    .withColumn('purchases', F.regexp_replace(F.col('purchases'),',','')) \
    .withColumn('purchases', F.round(F.col('purchases').cast('int')/100, 0)) \
    .withColumn('revenu', F.col('purchases')*(F.col('price').cast('int'))) \
    .withColumn('genre', F.explode(F.split(F.col('genre'),', '))) \
    .groupBy('genre').avg('revenu') \
    .withColumn('avg(revenu)', F.round(F.col('avg(revenu)'),0)) \
    .orderBy(F.col('avg(revenu)').desc()) \
    .collect() \
    )

genre,avg(revenu)
Massively Multiplayer,5594243.0
RPG,3951244.0
Action,3433799.0
Strategy,2564267.0
Photo Editing,2469359.0
Adventure,2420446.0
Simulation,2410231.0
,1904942.0
Racing,1794531.0
Web Publishing,1713561.0


Databricks visualization. Run in Databricks to view.

# Are most games available on Windows/Mac/Linux instead?
Windows is the most common platform

In [ ]:
platforms = ['platforms_mac', 'platforms_linux', 'platforms_windows']

[flat_data_no_tags.groupBy(platform).count().show() for platform in platforms]

+-------------+-----+
|platforms_mac|count|
+-------------+-----+
|         true|12770|
|        false|42921|
+-------------+-----+

+---------------+-----+
|platforms_linux|count|
+---------------+-----+
|           true| 8458|
|          false|47233|
+---------------+-----+

+-----------------+-----+
|platforms_windows|count|
+-----------------+-----+
|             true|55676|
|            false|   15|
+-----------------+-----+

Out[282]: [None, None, None]

# Do certain genres tend to be preferentially available on certain platforms?
Yes

In [ ]:
prefered_platform = flat_data_no_tags.withColumn('genre', F.explode(F.split(F.col('genre'),', '))) \
    .groupBy('genre').agg(F.sum(F.col('platforms_mac').cast('int')).alias('mac'),F.sum(F.col('platforms_linux').cast('int')).alias('linux'),F.sum(F.col('platforms_windows').cast('int')).alias('windows')) \
    .withColumn('total', F.col('mac')+F.col('linux')+F.col('windows')) \
    .withColumn('mac', F.round(F.col('mac')/F.col('total'),2) > 0.8) \
    .withColumn('linux', F.round(F.col('linux')/F.col('total'),2) > 0.8) \
    .withColumn('windows', F.round(F.col('windows')/F.col('total'),2) > 0.8)

prefered_platform.filter(F.col('mac') == True).show()
prefered_platform.filter(F.col('linux') == True).show()
prefered_platform.filter(F.col('windows') == True).show()

+-----+---+-----+-------+-----+
|genre|mac|linux|windows|total|
+-----+---+-----+-------+-----+
+-----+---+-----+-------+-----+

+-----+---+-----+-------+-----+
|genre|mac|linux|windows|total|
+-----+---+-----+-------+-----+
+-----+---+-----+-------+-----+

+-----------------+-----+-----+-------+-----+
|            genre|  mac|linux|windows|total|
+-----------------+-----+-----+-------+-----+
|        Education|false|false|   true|  392|
| Video Production|false|false|   true|  282|
|Software Training|false|false|   true|  203|
|    Photo Editing|false|false|   true|  125|
|        Utilities|false|false|   true|  832|
|            Movie|false|false|   true|    1|
+-----------------+-----+-----+-------+-----+



# Exploding CATEGORIES from flat_data_no_tags

In [ ]:
df_with_categories = flat_data_no_tags.withColumn("categories", F.explode('categories'))

In [ ]:
df_with_categories.select("categories").groupBy("categories").count().orderBy(F.col("count").desc()).toPandas()

,categories,count
0,Single-player,52025
1,Steam Achievements,27394
2,Steam Cloud,14235
3,Full controller support,11879
4,Multi-player,11455
5,Steam Trading Cards,9208
6,Partial Controller Support,7867
7,PvP,7070
8,Co-op,5616
9,Steam Leaderboards,5509


# Exploding TAGS from DATA_JSON

In [ ]:
tags_list = data_json.schema.jsonValue()["fields"][0]['type']['fields'][19]['type']['fields']
tags_name_list = [item['name'] for item in tags_list]
len(tags_name_list)

Out[2]: 441

In [ ]:
tags = data_json.withColumn('tags', F.array([]))
for item in tags_name_list:
    tags = tags.withColumn(item, F.col("data.tags").getField(item))
tags = tags.drop("data")
tags.printSchema()

root
 |-- id: string (nullable = true)
 |-- tags: array (nullable = false)
 |    |-- element: void (containsNull = false)
 |-- 1980s: long (nullable = true)
 |-- 1990's: long (nullable = true)
 |-- 2.5D: long (nullable = true)
 |-- 2D: long (nullable = true)
 |-- 2D Fighter: long (nullable = true)
 |-- 2D Platformer: long (nullable = true)
 |-- 360 Video: long (nullable = true)
 |-- 3D: long (nullable = true)
 |-- 3D Fighter: long (nullable = true)
 |-- 3D Platformer: long (nullable = true)
 |-- 3D Vision: long (nullable = true)
 |-- 4 Player Local: long (nullable = true)
 |-- 4X: long (nullable = true)
 |-- 6DOF: long (nullable = true)
 |-- 8-bit Music: long (nullable = true)
 |-- ATV: long (nullable = true)
 |-- Abstract: long (nullable = true)
 |-- Action: long (nullable = true)
 |-- Action RPG: long (nullable = true)
 |-- Action RTS: long (nullable = true)
 |-- Action Roguelike: long (nullable = true)
 |-- Action-Adventure: long (nullable = true)
 |-- Addictive: long (nullable = tr

In [ ]:
# Let's clean columns with names containing a '.'
tags = tags.withColumn('2_5D', F.col('`2.5D`')).drop(F.col('`2.5D`'))

## it seems like tags are customer added and their values is the number of customer "upvote" for a tag.
The counts are not very useful at this stage. I should start creating a single tags column with an array of notNull tags

In [ ]:
# Let's build our 'tags' column array
tags_to_list = []
for tag in tags.columns:
    if( (tag != 'id' and tag != 'tags') ):
        tags_to_list.append(F.when(F.col(tag).isNotNull(), F.lit(tag)))
tags = tags.withColumn('tags', F.array(*tags_to_list))
saved_tags = tags.select('id','tags').persist()

+--------------------+-----+
|                tags|count|
+--------------------+-----+
|[null, null, null...|  362|
|[null, null, null...|  315|
|[null, null, null...|  308|
|[null, null, null...|  301|
|[null, null, null...|  212|
|[null, null, null...|  135|
|[null, null, null...|  132|
|[null, null, null...|  124|
|[null, null, null...|  121|
|[null, null, null...|  113|
|[null, null, null...|  112|
|[null, null, null...|  105|
|[null, null, null...|  101|
|[null, null, null...|   85|
|[null, null, null...|   77|
|[null, null, null...|   66|
|[null, null, null...|   62|
|[null, null, null...|   60|
|[null, null, null...|   51|
|[null, null, null...|   49|
+--------------------+-----+
only showing top 20 rows



In [ ]:
saved_tags.withColumn('tags', F.explode(F.col('tags'))).filter(F.col('tags').isNotNull()).groupBy('tags').count().orderBy(F.col("count").desc()).show()

+--------------+-----+
|          tags|count|
+--------------+-----+
|         Indie|36062|
|  Singleplayer|27344|
|        Action|25022|
|        Casual|23993|
|     Adventure|23150|
|            2D|14547|
|      Strategy|11849|
|    Simulation|11599|
|           RPG|10162|
|        Puzzle| 9719|
|   Atmospheric| 9232|
|  Early Access| 7629|
|Pixel Graphics| 7521|
|            3D| 7430|
|    Story Rich| 7359|
|   Multiplayer| 6994|
|      Colorful| 6974|
|        Arcade| 6516|
|          Cute| 6431|
|  First-Person| 6155|
+--------------+-----+
only showing top 20 rows



In [ ]:
# Let's create a clean list of tags for each id
tags_df = saved_tags.withColumn('tags', F.explode(F.col('tags'))).filter(F.col('tags').isNotNull()).groupBy('id').agg(F.collect_list(F.col('tags')).alias('tags'))
tags_df.count()

Out[42]: 55556

In [ ]:
# Let's merge our clean tags to the flat_data_no_tags dataframe to get a full cleaned dataframe
flat_data_with_tags = flat_data_no_tags.join(tags_df, 'id')
flat_data_with_tags.count()

Out[47]: 55556

In [ ]:
# Let's check the avg price per tag
display(flat_data_with_tags.select('id','tags',F.col("price").cast('float').alias("price")) \
    .withColumn('tags', F.explode(F.col('tags'))) \
    .groupBy('tags').avg("price") \
    .orderBy(F.col('avg(price)').desc()) \
    .withColumn("avg_price", F.round(F.col('avg(price)')/100, 0 )) \
    .drop('avg(price)') \
    .collect() \
    )

tags,avg_price
Musou,48.0
Rugby,28.0
Web Publishing,24.0
TrackIR,21.0
360 Video,20.0
Warhammer 40K,19.0
Photo Editing,19.0
Audio Production,19.0
Software Training,19.0
Video Production,18.0


Databricks visualization. Run in Databricks to view.